In [ ]:
'''
class Node:
    '''
    Defines each node of the Decision Tree
    '''
    def __init__(self, feature=None, left=None, right=None, threshold=None, value=None):
        self.feature = feature
        self.left = left
        self.right = right
        self.threshold = threshold
        
        # for leaf node
        self.value = value
    
    
class DTClassifier:
    '''
    Decision Tree Classifier
    '''
    def __init__(self, max_depth=3, min_sample_split=2):
        self.max_depth = max_depth
        self.min_sample_split = min_sample_split
        
        self.root_node = None
        
        
    def build_tree(self, train_X, train_y, current_depth=0):
        num_datapoints = len(train_X)
        
        if (num_datapoints >= self.min_sample_split) and (current_depth <= self.max_depth):
            split = self.cont_to_cat(train_X, train_y)
            
            if split['info_gain'] > 0:
                left_branch = self.build_tree(split['left_branch_X'], split['left_branch_y'], current_depth+1)
                right_branch = self.build_tree(split['right_branch_X'], split['right_branch_y'], current_depth+1)
                
                return Node(feature=split['feature'],
                            left=left_branch, right=right_branch, 
                            threshold=split['threshold'])
                
        # in case of leaf node
        leaf_value = max(train_y)
        return Node(value=leaf_value)
    
    
    def cont_to_cat(self, train_X, train_y):
        '''
        Makes the best possible split based on the information gain (entropy loss) 
        '''
        split = {}
        info_gain = -1
        
        features = train_X.columns.tolist()
        for Feature in features:
            possible_thresholds = train_X[Feature].unique()
            for Threshold in possible_thresholds:
                # left branch
                left_branch_X = train_X[train_X[Feature] <= Threshold]
                left_branch_y = train_y[train_X[Feature] <= Threshold]
                
                # right branch
                right_branch_X = train_X[train_X[Feature] > Threshold]
                right_branch_y = train_y[train_X[Feature] > Threshold]
                
                if len(left_branch_X > 0) and len(right_branch_X > 0):
                    current_info_gain = self.information_gain(train_y, left_branch_y, right_branch_y)
                    
                    if current_info_gain > info_gain:
                        split['feature'] = Feature
                        split['threshold'] = Threshold
                        split['left_branch_X'] = left_branch_X
                        split['left_branch_y'] = left_branch_y
                        split['right_branch_X'] = right_branch_X
                        split['right_branch_y'] = right_branch_y
                        split['info_gain'] = current_info_gain
                        info_gain = current_info_gain
        
        return split
    
    
    def train(self, train_X, train_y):
        self.root_node = self.build_tree(train_X, train_y)
    
    
    def predict(self, test_X, node):
        '''
        Predicts the class for a single data point
        (Recursive Function)
        '''
        if node.value != None:
            return node.value
        
        node_feature = test_X[node.feature]
        if node_feature <= node.threshold:
            return self.predict(test_X, node.left)
        else:
            return self.predict(test_X, node.right)
        
        
    def test(self, test_X, test_y):
        '''
        Returns the predictions and accuracy scored for the test dataset
        '''
        pred_y = []
        Test_y = test_y.tolist()
        for x in range(len(test_X)):
            pred_y.append(self.predict(test_X.iloc[x], self.root_node))
            
        print('True Values:', Test_y)
        print('Predictions:', pred_y)
        
        accuracy = 0
        for each in range(len(pred_y)):
            if pred_y[each] == Test_y[each]:
                accuracy += 1
        accuracy /= len(test_X)
        print('Accuracy:', accuracy)
        
        classwise_accuracy = 0
        classes = test_y.unique()
        classwise_true_pos_count = {}
        for Class in classes:
            classwise_true_pos_count[Class] = 0
            
        classwise_count = test_y.value_counts().to_dict()
        
        for each in range(len(pred_y)):
            if pred_y[each] == Test_y[each]:
                Class = Test_y[each]
                classwise_true_pos_count[Class] += 1
        
        for Class in classes:
            classwise_accuracy += classwise_true_pos_count[Class]/classwise_count[Class]
        
        classwise_accuracy /= len(classes)
        print('Classwise Accuracy:', classwise_accuracy)
    
    
    def information_gain(self, y_node, y_left_branch, y_right_branch):
        total_datapoints = len(y_node)
        left_branch_weight = len(y_left_branch)/total_datapoints 
        right_branch_weight = (len(y_right_branch))/total_datapoints
        
        return self.entropy_loss(y_node) - \
            ( left_branch_weight*self.entropy_loss(y_left_branch) + right_branch_weight*self.entropy_loss(y_right_branch) )
        
        
    def entropy_loss(self, y):
        probs = y.value_counts().to_numpy()/len(y)
        entropy = 0
        for each in probs:
            entropy += each * math.log2(each)

        return -entropy
'''

In [ ]:
'''
model = DTClassifier()
model.train(train_X, train_y)
model.test(test_X, test_y)
'''